In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from os import listdir
import glob
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from __future__ import print_function
import os
from scipy import misc
import glob
import matplotlib.pyplot as plt   
import sys 
from scipy.ndimage import rotate
#from scipy.misc import imread, imshow
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import sys
import shutil
import random
import numpy as np
from keras.models import load_model
import pandas as pd
import cv2
from tensorflow.contrib.keras.api.keras.layers import Dropout
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Conv2D
from tensorflow.contrib.keras.api.keras.layers import MaxPooling2D
from tensorflow.contrib.keras.api.keras.layers import Flatten
from tensorflow.contrib.keras.api.keras.layers import Dense
from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
import os

Using TensorFlow backend.


In [2]:
!unzip "/content/Data_Cars and Trucks.zip"

Archive:  /content/Data_Cars and Trucks.zip
   creating: Data_Cars and Trucks/car/
  inflating: Data_Cars and Trucks/car/100171.jpg  
  inflating: Data_Cars and Trucks/car/102685.jpg  
  inflating: Data_Cars and Trucks/car/103773.jpg  
  inflating: Data_Cars and Trucks/car/111104.jpg  
  inflating: Data_Cars and Trucks/car/115988.jpg  
  inflating: Data_Cars and Trucks/car/119092.jpg  
  inflating: Data_Cars and Trucks/car/128887.jpg  
  inflating: Data_Cars and Trucks/car/130726.jpg  
  inflating: Data_Cars and Trucks/car/13193.jpg  
  inflating: Data_Cars and Trucks/car/137920.jpg  
  inflating: Data_Cars and Trucks/car/139345.jpg  
  inflating: Data_Cars and Trucks/car/141101.jpg  
  inflating: Data_Cars and Trucks/car/143302.jpg  
  inflating: Data_Cars and Trucks/car/145185.jpg  
  inflating: Data_Cars and Trucks/car/145195.jpg  
  inflating: Data_Cars and Trucks/car/145512.jpg  
  inflating: Data_Cars and Trucks/car/146072.jpg  
  inflating: Data_Cars and Trucks/car/153019.jpg  


In [0]:
folder_path = '/content/Data_Cars and Trucks'

In [0]:
trucks_cars=os.listdir(folder_path)

In [5]:
trucks_cars

['car', 'truck']

In [0]:
trucks = os.listdir(folder_path+'/'+trucks_cars[1])

In [0]:
cars = os.listdir(folder_path+'/'+trucks_cars[0])

In [0]:
cars

In [11]:
len(trucks),len(cars)

(402, 392)

In [0]:
trucks_train = trucks[:316]
cars_train = cars[:316]
trucks_test = trucks[316:392]
cars_test = cars[316:392]

In [19]:
os.mkdir(folder_path+'/train')
os.mkdir(folder_path+'/test')

FileExistsError: ignored

In [0]:
os.mkdir(folder_path+'/train'+'/cars')
os.mkdir(folder_path+'/train'+'/trucks')
os.mkdir(folder_path+'/test'+'/cars')
os.mkdir(folder_path+'/test'+'/trucks')

In [0]:
for i in range(0,len(trucks_train)):
    image = cv2.imread(folder_path+"/"+trucks_cars[1]+'/'+trucks_train[i]) 
    cv2.imwrite(os.path.join(folder_path+'/train'+'/trucks'+'/'+trucks_train[i]),image)

In [0]:
for j in range(0,len(cars_train)):
    image = cv2.imread(folder_path+"/"+trucks_cars[0]+'/'+cars_train[j])
    cv2.imwrite(os.path.join(folder_path+'/train'+'/cars'+'/'+cars_train[j]),image)

In [0]:
for i in range(0,len(trucks_test)):
    image = cv2.imread(folder_path+"/"+trucks_cars[1]+'/'+trucks_test[i]) 
    cv2.imwrite(os.path.join(folder_path+'/test'+'/trucks'+'/'+trucks_test[i]),image)
    image1 = cv2.imread(folder_path+"/"+trucks_cars[0]+'/'+cars_test[i]) 
    cv2.imwrite(os.path.join(folder_path+'/test'+'/cars'+'/'+cars_test[i]),image1)

In [0]:
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n"\
            .format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1
 
    def on_train_begin(self, logs={}):
        self.losses += 'Training begins...\n'

In [0]:
training_set_path = os.path.join('/content/Data_Cars and Trucks/train')
test_set_path = os.path.join('/content/Data_Cars and Trucks/test')

In [0]:
# Initialising the CNN
classifier = Sequential()
 
# Step 1 - Convolution
input_size = (192, 192)
classifier.add(Conv2D(32, (3, 3), input_shape=(*input_size, 3), activation='relu'))
 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))  # 2x2 is optimal
 
# classifier.add(Conv2D(32, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# classifier.add(Conv2D(64, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
# classifier.add(Conv2D(64, (3, 3), activation='relu'))
# classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=2, activation='softmax'))
 
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 

In [29]:
# Part 2 - Fitting the CNN to the images
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
 
test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            class_mode='categorical')
 

Found 632 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


In [30]:
# Create a loss history
history = LossHistory()
 
classifier.fit_generator(training_set,
                         steps_per_epoch=632,
                         epochs=10,
                         validation_data=test_set,
                         validation_steps=152,
                         workers=12,
                         callbacks=[history])
 

Epoch 1/10
632/632 [==============================] - 235s 372ms/step - loss: 0.4886 - acc: 0.7540 - val_loss: 1.0194 - val_acc: 0.6218
Epoch 2/10
632/632 [==============================] - 234s 370ms/step - loss: 0.1822 - acc: 0.9269 - val_loss: 1.9470 - val_acc: 0.6274
Epoch 3/10
632/632 [==============================] - 231s 365ms/step - loss: 0.1008 - acc: 0.9639 - val_loss: 1.9852 - val_acc: 0.6927
Epoch 4/10
632/632 [==============================] - 234s 370ms/step - loss: 0.0697 - acc: 0.9759 - val_loss: 2.1779 - val_acc: 0.6541
Epoch 5/10
632/632 [==============================] - 231s 366ms/step - loss: 0.0550 - acc: 0.9802 - val_loss: 2.4161 - val_acc: 0.6992
Epoch 6/10
632/632 [==============================] - 231s 366ms/step - loss: 0.0481 - acc: 0.9833 - val_loss: 2.2629 - val_acc: 0.6592
Epoch 7/10
632/632 [==============================] - 233s 368ms/step - loss: 0.0330 - acc: 0.9893 - val_loss: 2.3566 - val_acc: 0.6869
Epoch 8/10
632/632 [============================